# Neural Network
In this notebook, we will employ a neural network in order to attempt to obtain a greater accuracy on our test set for our data.

In [1]:
#import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import ParameterGrid

#Neural Network libraries:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

2023-10-09 18:31:45.096790: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 18:31:45.124136: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 18:31:45.124600: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 18:31:45.634356: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('clean_data.csv')
df.head()

,SIZE_BINARY_CLASS,FIRE_SIZE_CLASS,FIRE_SIZE,FIRE_YEAR,FIRE_DOY,LATITUDE,LONGITUDE,DISCOVERY_TIME_Evening,DISCOVERY_TIME_Midday,DISCOVERY_TIME_Night or Morning,CONT_TIME_Evening,CONT_TIME_Morning,CONT_TIME_Night or Midday,NWCG_CAUSE_CLASSIFICATION_Human,NWCG_CAUSE_CLASSIFICATION_Natural,OWNER_DESCR_Government,OWNER_DESCR_Private,REGION_East,REGION_Midwest,REGION_West
0,Small,A,0.10,2005,33,40.036944,-121.005833,0,1,0,1,0,0,1,0,1,0,0,0,1
1,Small,A,0.25,2004,133,38.933056,-120.404444,0,0,1,0,0,1,0,1,1,0,0,0,1
2,Small,A,0.10,2004,152,38.984167,-120.735556,1,0,0,1,0,0,1,0,0,0,0,0,1
3,Small,A,0.10,2004,182,38.559167,-119.913333,0,1,0,0,0,1,0,1,1,0,0,0,1
4,Small,A,0.10,2004,182,38.559167,-119.933056,0,1,0,0,0,1,0,1,1,0,0,0,1


In [3]:
df.shape

(2299932, 20)

In [4]:
df.columns

Index(['SIZE_BINARY_CLASS', 'FIRE_SIZE_CLASS', 'FIRE_SIZE', 'FIRE_YEAR',
       'FIRE_DOY', 'LATITUDE', 'LONGITUDE', 'DISCOVERY_TIME_Evening',
       'DISCOVERY_TIME_Midday', 'DISCOVERY_TIME_Night or Morning',
       'CONT_TIME_Evening', 'CONT_TIME_Morning', 'CONT_TIME_Night or Midday',
       'NWCG_CAUSE_CLASSIFICATION_Human', 'NWCG_CAUSE_CLASSIFICATION_Natural',
       'OWNER_DESCR_Government', 'OWNER_DESCR_Private', 'REGION_East',
       'REGION_Midwest', 'REGION_West'],
      dtype='object')

In [5]:
X = df[['FIRE_YEAR', 'FIRE_DOY', 'LATITUDE', 'LONGITUDE', 'DISCOVERY_TIME_Evening', \
        'DISCOVERY_TIME_Midday', 'DISCOVERY_TIME_Night or Morning', 'CONT_TIME_Evening',\
        'CONT_TIME_Morning', 'CONT_TIME_Night or Midday', 'NWCG_CAUSE_CLASSIFICATION_Human',\
        'NWCG_CAUSE_CLASSIFICATION_Natural', 'OWNER_DESCR_Government', 'OWNER_DESCR_Private',\
        'REGION_East', 'REGION_Midwest', 'REGION_West']]
y = df['SIZE_BINARY_CLASS']

In [6]:
#sample from dataframe - s for sample
sdf = df.sample(frac=.05, random_state=11)
sdf['SIZE_BINARY_CLASS'].value_counts(normalize=True)

SIZE_BINARY_CLASS
Small    0.505109
Large    0.494891
Name: proportion, dtype: float64

In [7]:
sdf.shape

(114997, 20)

In [8]:
# X1 and y1 for samples!
X1 = sdf.drop(['SIZE_BINARY_CLASS', 'FIRE_SIZE_CLASS', 'FIRE_SIZE'], axis=1)
y1 = sdf['SIZE_BINARY_CLASS']
X1.head()

,FIRE_YEAR,FIRE_DOY,LATITUDE,LONGITUDE,DISCOVERY_TIME_Evening,DISCOVERY_TIME_Midday,DISCOVERY_TIME_Night or Morning,CONT_TIME_Evening,CONT_TIME_Morning,CONT_TIME_Night or Midday,NWCG_CAUSE_CLASSIFICATION_Human,NWCG_CAUSE_CLASSIFICATION_Natural,OWNER_DESCR_Government,OWNER_DESCR_Private,REGION_East,REGION_Midwest,REGION_West
751212,2001,135,44.524980,-103.457020,0,0,0,0,0,0,1,0,0,1,0,1,0
1783988,2014,281,33.500700,-112.289800,1,0,0,1,0,0,0,0,0,0,0,0,1
813663,2005,289,31.060600,-90.411400,0,0,0,0,0,0,1,0,0,0,0,0,0
501812,1998,101,46.032830,-94.350666,0,0,0,0,0,0,1,0,0,1,0,1,0
1547576,2011,56,28.840313,-98.599548,0,0,0,0,0,0,1,0,0,0,0,0,0


In [9]:
X1.shape

(114997, 17)

In [10]:
y1.shape

(114997,)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=11)

In [12]:
#splitting training data into training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train, random_state=11)

In [13]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.3, stratify=y1, random_state=11)

In [14]:
#splitting training data into training and validation
X_train1, X_val1, y_train1, y_val1 = train_test_split(X_train1, y_train1, test_size=0.3, stratify=y_train1, random_state=11)

In [15]:
#sample from dataframe - s for sample
sdf1 = df.sample(frac=.01, random_state=11)
sdf1['SIZE_BINARY_CLASS'].value_counts(normalize=True)

SIZE_BINARY_CLASS
Small    0.505022
Large    0.494978
Name: proportion, dtype: float64

In [16]:
X2 = sdf1.drop(['SIZE_BINARY_CLASS', 'FIRE_SIZE_CLASS', 'FIRE_SIZE'], axis=1)
y2 = sdf1['SIZE_BINARY_CLASS']
X2.head()

,FIRE_YEAR,FIRE_DOY,LATITUDE,LONGITUDE,DISCOVERY_TIME_Evening,DISCOVERY_TIME_Midday,DISCOVERY_TIME_Night or Morning,CONT_TIME_Evening,CONT_TIME_Morning,CONT_TIME_Night or Midday,NWCG_CAUSE_CLASSIFICATION_Human,NWCG_CAUSE_CLASSIFICATION_Natural,OWNER_DESCR_Government,OWNER_DESCR_Private,REGION_East,REGION_Midwest,REGION_West
751212,2001,135,44.524980,-103.457020,0,0,0,0,0,0,1,0,0,1,0,1,0
1783988,2014,281,33.500700,-112.289800,1,0,0,1,0,0,0,0,0,0,0,0,1
813663,2005,289,31.060600,-90.411400,0,0,0,0,0,0,1,0,0,0,0,0,0
501812,1998,101,46.032830,-94.350666,0,0,0,0,0,0,1,0,0,1,0,1,0
1547576,2011,56,28.840313,-98.599548,0,0,0,0,0,0,1,0,0,0,0,0,0


In [17]:
X2.shape

(22999, 17)

In [18]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.3, stratify=y2, random_state=11)
#splitting training data into training and validation
X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train2, y_train2, test_size=0.3, stratify=y_train2, random_state=11)

## Multilayer Perceptron
For our attempt at using deep learning to predict wildfire size, we are going to use a multi-layer perceptron feed-forward neural network. This will be built using keras and have a relatively simple structure since our data is not overly complex. The input layer will include our features (and therefore have 17 neurons in it). The model will have 5 hidden layers with 16 nodes each, all of them using a relu activation function. Finally, the output layer will have a single node (since it is a binary classification problem) using a sigmoid function. This network architecture is complex enough to handle the size of the dataset, while simple enough that overfitting will not likely be a problem. This architecture is an initial estimate of the best architecture, and future work will optimize the structure of the neural network. 

Since Keras requires the output variable to be in a numeric format, I encode our target variable below.

In [19]:
# Create a label encoder
label_encoder = LabelEncoder()

# Fit the encoder on the original class labels and transform them
y_train2_encoded = label_encoder.fit_transform(y_train2)
y_test2_encoded = label_encoder.transform(y_test2)
y_val2_encoded = label_encoder.transform(y_val2)

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_val_encoded = label_encoder.transform(y_val)

In [20]:
# Create a new sequential model
model = keras.Sequential()
model.add(layers.Input(shape=(17)))

# Declare the hidden layers
# five hidden layers with 16 nodes each
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(16, activation="relu"))

# Declare the output layer
model.add(layers.Dense(1, activation="sigmoid"))

model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),  
    # Loss function to minimize
    loss=keras.losses.BinaryCrossentropy(),
    # Metric used to evaluate model
    metrics=['accuracy']
)

history = model.fit(X_train, y_train_encoded, epochs=100, verbose=2)

2023-10-09 18:37:44.137984: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-09 18:37:44.138602: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/100
35218/35218 - 20s - loss: 0.6479 - accuracy: 0.6376 - 20s/epoch - 564us/step
Epoch 2/100
35218/35218 - 19s - loss: 0.6255 - accuracy: 0.6699 - 19s/epoch - 548us/step
Epoch 3/100
35218/35218 - 19s - loss: 0.6148 - accuracy: 0.6775 - 19s/epoch - 551us/step
Epoch 4/100
35218/35218 - 19s - loss: 0.6062 - accuracy: 0.6827 - 19s/epoch - 551us/step
Epoch 5/100
35218/35218 - 19s - loss: 0.5996 - accuracy: 0.6869 - 19s/epoch - 544us/step
Epoch 6/100
35218/35218 - 19s - loss: 0.5964 - accuracy: 0.6880 - 19s/epoch - 547us/step
Epoch 7/100
35218/35218 - 20s - loss: 0.5942 - accuracy: 0.6892 - 20s/epoch - 557us/step
Epoch 8/100
35218/35218 - 20s - loss: 0.5928 - accuracy: 0.6900 - 20s/epoch - 567us/step
Epoch 9/100
35218/35218 - 18s - loss: 0.5909 - accuracy: 0.6913 - 18s/epoch - 523us/step
Epoch 10/100
35218/35218 - 20s - loss: 0.5900 - accuracy: 0.6914 - 20s/epoch - 560us/step
Epoch 11/100
35218/35218 - 19s - loss: 0.5886 - accuracy: 0.6926 - 19s/epoch - 552us/step
Epoch 12/100
35218/

35218/35218 - 19s - loss: 0.5683 - accuracy: 0.7088 - 19s/epoch - 548us/step
Epoch 93/100
35218/35218 - 19s - loss: 0.5689 - accuracy: 0.7082 - 19s/epoch - 549us/step
Epoch 94/100
35218/35218 - 20s - loss: 0.5699 - accuracy: 0.7062 - 20s/epoch - 558us/step
Epoch 95/100
35218/35218 - 19s - loss: 0.5701 - accuracy: 0.7063 - 19s/epoch - 543us/step
Epoch 96/100
35218/35218 - 20s - loss: 0.5696 - accuracy: 0.7072 - 20s/epoch - 555us/step
Epoch 97/100
35218/35218 - 19s - loss: 0.5696 - accuracy: 0.7080 - 19s/epoch - 542us/step
Epoch 98/100
35218/35218 - 19s - loss: 0.5703 - accuracy: 0.7068 - 19s/epoch - 542us/step
Epoch 99/100
35218/35218 - 20s - loss: 0.5698 - accuracy: 0.7071 - 20s/epoch - 555us/step
Epoch 100/100
35218/35218 - 19s - loss: 0.5703 - accuracy: 0.7070 - 19s/epoch - 547us/step


In [21]:
# Evaluate the network
train_accuracy = history.history["accuracy"][-1]
result = model.evaluate(X_val,y_val_encoded, verbose=0)
result_test = model.evaluate(X_test,y_test_encoded, verbose=0)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy: {result[1]:.4f}") 
print(f"Test Accuracy: {result_test[1]:.4f}")

Train Accuracy: 0.7070
Validation Accuracy: 0.7108
Test Accuracy: 0.7104


Our neural network yielded the best model yet! Our test accuracy was 71.4%, which is better than all of the traditional algorithms yielded. For future work, I would like to optimize the architecture and incorporate methods such as dropout in order to yield the highest accuracy possible in our network.